In [ ]:
import torch
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import skimage
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
import torchvision
train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1),
        torchvision.transforms.RandomAffine(15),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation(15),
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
valid_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((64, 64)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

In [ ]:
image_file_path = 'Downloads/archive/food41/images/'
train_dataset = torchvision.datasets.ImageFolder(root = image_file_path, transform = train_transforms)

In [ ]:
def show_transform_images(dataset):
    loader = torch.utils.data.DataLoader(dataset,batch_size = 6,shuffle=True)
    batch = next(iter(loader))
    images, labels = batch
    grid = torchvision.utils.make_grid(images,nrow = 1)
    plt.figure(figsize=(11,11))
    plt.imshow(np.transpose(grid,(1,2,0)))
    print('labels:',labels)

In [ ]:
from torch.utils.data import random_split
train_len = round(len(train_dataset )* 0.75)
test_len = round(len(train_dataset)* 0.25)


train_data, test_data = random_split(train_dataset, [train_len, test_len])

print("The length of train data is:",len(train_data))

print("The length of test data is:",len(test_data))



In [ ]:
from torch.utils.data import WeightedRandomSampler, DataLoader


In [ ]:
from collections import Counter
print(dict(Counter(train_dataset.targets)))

In [ ]:
import numpy as np 

y_train_indices = train_data.indices

y_train = [train_dataset.targets[i] for i in y_train_indices]

class_sample_count = np.array(
    [len(np.where(y_train == t)[0]) for t in np.unique(y_train)])

In [ ]:
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_train])
samples_weight = torch.from_numpy(samples_weight)

In [ ]:
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=128, sampler=sampler)
test_dataloader = DataLoader(test_data, batch_size=128)

In [ ]:
def set_device():
    if torch.cuda.is_available():
        dev="cuda:0"
    else:
        dev="cpu"
    return torch.device(dev)

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    for epoch in range(n_epochs):
        print(f"Epoch number: {epoch + 1}")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images,labels = data
            images = images.to(device)
            labels = labels.to(device)
            total+=labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            
            _,predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs,labels)
            
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            
            running_correct += (labels == predicted).sum().item()
            
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.00 * (running_correct/total)
        
        print(f"    - Training dataset. Got {running_correct} out of {total} images correctly {epoch_acc}%. Epoch loss {epoch_loss}")
        
        evaluate_model_on_test_set(model, test_dataloader)
        
    print("Finished")
    return model
    

In [ ]:
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
            
    epoch_acc = 100.0 * predicted_correctly_on_epoch / total
    print(f" - Testing dataset Got {predicted_correctly_on_epoch} out of {total} images correct. {epoch_acc}%")

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


resnet18_model = models.resnet18(pretrained=False)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 4
resnet18_model.fc = nn.Linear(num_ftrs,number_of_classes)
device = set_device()
resnet18_model = resnet18_model.to(device)
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.SGD(resnet18_model.parameters(), lr = 0.01, momentum = 0.9, weight_decay = 0.003)



In [ ]:
train_nn(resnet18_model, train_dataloader, test_dataloader, loss_fn, optimizer, 10)

In [ ]:
print(resnet18_model)

In [ ]:
import pickle

In [ ]:
filename = 'resnet18_torch_trained_model.sav'

In [ ]:
pickle.dump(resnet18_model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
def pre_image(image_path,model):
   img = Image.open(image_path)
   mean = [0.485, 0.456, 0.406] 
   std = [0.229, 0.224, 0.225]
   transform_norm = transforms.Compose([transforms.ToTensor(), 
   transforms.Resize((224,224)),transforms.Normalize(mean, std)])
   # get normalized image
   img_normalized = transform_norm(img).float()
   img_normalized = img_normalized.unsqueeze_(0)
   # input = Variable(image_tensor)
   img_normalized = img_normalized.to(device)
   # print(img_normalized.shape)
   with torch.no_grad():
      model.eval()  
      output = model(img_normalized)
     # print(output)
      index = output.data.cpu().numpy().argmax()
      classes = train_dataset.classes
      m = nn.Softmax(dim=1)
      results = m(output)
      results = sorted(results)
      class_name = classes[index]
      result_name = f'{class_name}, Confidence Value: {round(torch.max(results[0]).item() * 100, 2)} %'
      return result_name

In [ ]:
pre_image('/Users/aadrijupadya/Downloads/test.jpeg', loaded_model)

In [ ]:
train_dataset.class_to_idx

In [ ]:
input = torch.randn(2, 3)
output = m(input)

In [ ]:
%history


print(str(input[0][1].item()))